In [2]:
import json
train = []
with open('train') as f:
    for line in f:
        train.append(json.loads(line.strip()))
val = []
with open('val') as f:
    for line in f:
        val.append(json.loads(line.strip()))
with open('test_no_reference') as f:
  test = f.readlines()

In [2]:
print(train[:3])

[{'dst': '- Intriguing.', 'src': '◄▴◓◠▨ ◨▽◠▦◈◬◓▪▼◬▵'}, {'dst': 'He would need to repeat his vows in the land of the living and drink from the wine of ages.', 'src': '▽◪◎◗▦◫▦◫ ▫▴▨◓◠◓ ▴▫◎◪▱◫ ◚▴ ◞◧▦◞▾▢▱◨▨ ◒◠◓◠◀▪▦◈◠▦ ◫◉◎▴▱◫▵'}, {'dst': "You couldn't even answer my texts?", 'src': '◄▴◞◠▸▱◠◓▪◎◠ ◀◫▱◪ ▼◪◚◠▻ ◚▴◓▴◎◪◈◗▦ ◎◫?'}]


In [3]:
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from torch import nn

from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
import sys
import numpy as np
import torch
from tqdm.notebook import tqdm
from torch.optim import Adam
import os
import subprocess

from matplotlib import pyplot as plt
from datetime import datetime
from torch.utils.tensorboard import SummaryWriter
import shutil
import random
import itertools
from datasets import load_metric

bleu_metric = load_metric("bleu")

C:\Users\Katie\AppData\Local\Temp\ipykernel_24380\1372133618.py:22: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  bleu_metric = load_metric("bleu")
c:\Users\Katie\miniconda3\envs\mltest\Lib\site-packages\datasets\load.py:759: FutureWarning: The repository for bleu contains custom code which must be executed to correctly load the metric. You can inspect the repository content at https://raw.githubusercontent.com/huggingface/datasets/2.19.1/metrics/bleu/bleu.py
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this metric from the next major release of `datasets`.
  warnings.warn(


In [4]:
dst=[]
src=[]
for i in range(len(train)):
    dst.append(train[i]['dst'])
    src.append(train[i]['src'])


In [5]:
src
with open('src', 'w') as writer:
    for i in src:
        writer.write(i)

In [6]:
from datasets import load_dataset
train_zet = load_dataset('json', data_files='train', split="train")

In [7]:
train_zet['dst']

['- Intriguing.',
 'He would need to repeat his vows in the land of the living and drink from the wine of ages.',
 "You couldn't even answer my texts?",
 'How fast do you go?',
 "He's talking about a few right here in Lisbon, mostly Jews who have escaped the Germans, that's all.",
 'For instance, Ben is not a guy who skips work.',
 'Goodbye, Jakob.',
 'Sure beats the foam domes round here.',
 'Oh, wait a second!',
 'The time is now.',
 'Our creditors are voting with their pocketbook by buying up our debt.',
 'Over there.',
 "She-- she's the purple sandpiper.",
 "I'm meeting a friend.",
 'To ensure peace between the two royal households.',
 'I worked even if the boss kicked or slapped me.',
 'I could eat even you, Super Mario.',
 "But there's 12 of them.",
 "She's the queen of the Golden City",
 "You don't see me shirking.",
 "I'm just saying, like, we're making a good show, but, you know...",
 "And now I'm all taking care of, but what are you going to do about him?",
 '- We gotta work 

In [8]:
from tokenizers import (
    decoders,
    models,
    normalizers,
    pre_tokenizers,
    processors,
    trainers,
    Tokenizer,
)
tokenizer = Tokenizer(models.Unigram())
from tokenizers import Regex

tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFKD(),
        normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),
    ]
)

In [9]:
tokenizer.pre_tokenizer = pre_tokenizers.Metaspace()

In [10]:
tokenizer.pre_tokenizer.pre_tokenize_str("Let's test the pre-tokenizer!")

[("▁Let's", (0, 5)),
 ('▁test', (5, 10)),
 ('▁the', (10, 14)),
 ('▁pre-tokenizer!', (14, 29))]

In [11]:
tokenizer.pre_tokenizer.pre_tokenize_str("◠▦◬▦■ ◉◗▢◕◗ ◍◗▱◎ ▽◠▽▪▦◠ ◕▴◉◗▦▼")

[('▁◠▦◬▦■', (0, 5)),
 ('▁◉◗▢◕◗', (5, 11)),
 ('▁◍◗▱◎', (11, 16)),
 ('▁▽◠▽▪▦◠', (16, 23)),
 ('▁◕▴◉◗▦▼', (23, 30))]

In [12]:
def get_training_corpus():
    for i in range(0, len(src)):
        yield src[i]

In [13]:
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=35000, special_tokens=special_tokens, unk_token="<unk>"
)
tokenizer.train_from_iterator(get_training_corpus(), trainer=trainer)

In [17]:
tokenizer.model = models.Unigram()
tokenizer.train(["src"], trainer=trainer)



In [18]:
encoding = tokenizer.encode("Let's test this tokenizer.")
print(encoding.ids)

[8, 2, 8356, 34900, 157, 2, 8, 34900, 8356, 2, 34900, 8, 34900, 2, 6144, 2, 8, 34900, 30407, 2, 8356, 34273, 6144, 2, 8356, 2]


In [19]:
tokenizer.decode([8, 17508, 7, 8941, 1, 0])

'▁ ▁◞◫▦◳◠▱ ■ ▁◂▱◈▾◐◨▦'

In [20]:
cls_token_id = tokenizer.token_to_id("<cls>")
sep_token_id = tokenizer.token_to_id("<sep>")
print(cls_token_id, sep_token_id)

0 1


In [21]:
tokenizer.post_processor = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)],
)

In [22]:
encoding = tokenizer.encode("◔▢◕◭▦◭◎■ ▷◠◳◬◓▵")
print(encoding.tokens)
print(encoding.type_ids)

['▁', '◔▢◕◭▦◭◎', '■', '▁▷◠◳◬◓▵', '<sep>', '<cls>']
[0, 0, 0, 0, 0, 2]


In [23]:
tokenizer.decoder = decoders.Metaspace()

In [24]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

c:\Users\Katie\miniconda3\envs\mltest\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [25]:
tokenizer.save("tokenizer_Unigram_zetas.json")

english

In [5]:
dst
with open('dst', 'w') as writer:
    for i in dst:
        writer.write(i)

In [6]:
def get_training_corpus():
    for i in range(0, len(dst)):
        yield dst[i]

In [10]:

tokenizer_eng = Tokenizer(models.Unigram())
from tokenizers import Regex

tokenizer.normalizer = normalizers.Sequence(
    [
        normalizers.Replace("``", '"'),
        normalizers.Replace("''", '"'),
        normalizers.NFKD(),
        normalizers.StripAccents(),
        normalizers.Replace(Regex(" {2,}"), " "),
    ]
)

In [ ]:
special_tokens = ["<cls>", "<sep>", "<unk>", "<pad>", "<mask>", "<s>", "</s>"]
trainer = trainers.UnigramTrainer(
    vocab_size=35000, special_tokens=special_tokens, unk_token="<unk>"
)
tokenizer_eng.train_from_iterator(get_training_corpus(), trainer=trainer)

: 

In [ ]:
tokenizer_eng.model = models.Unigram()
tokenizer_eng.train(["dst"], trainer=trainer)

In [ ]:
encoding = tokenizer_eng.encode("Let's test this tokenizer.")
print(encoding.tokens)

['▁Let', "'s", '▁test', '▁this', '▁token', 'ize', 'r.', '<sep>', '<cls>']


In [ ]:
cls_token_id = tokenizer_eng.token_to_id("<cls>")
sep_token_id = tokenizer_eng.token_to_id("<sep>")
print(cls_token_id, sep_token_id)

0 1


In [ ]:
tokenizer_eng.post_processor = processors.TemplateProcessing(
    single="$A:0 <sep>:0 <cls>:2",
    pair="$A:0 <sep>:0 $B:1 <sep>:1 <cls>:2",
    special_tokens=[("<sep>", sep_token_id), ("<cls>", cls_token_id)],
)

In [ ]:
encoding = tokenizer_eng.encode("Let's test this tokenizer...", "on a pair of sentences!")
print(encoding.tokens)
print(encoding.type_ids)

['▁Let', "'s", '▁test', '▁this', '▁token', 'ize', 'r', '...', '<sep>', '▁on', '▁a', '▁pair', '▁of', '▁sentence', 's', '!', '<sep>', '<cls>']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 2]


In [ ]:
tokenizer_eng.decoder = decoders.Metaspace()

In [36]:
from transformers import PreTrainedTokenizerFast

wrapped_tokenizer = PreTrainedTokenizerFast(
    tokenizer_object=tokenizer,
    bos_token="<s>",
    eos_token="</s>",
    unk_token="<unk>",
    pad_token="<pad>",
    cls_token="<cls>",
    sep_token="<sep>",
    mask_token="<mask>",
    padding_side="left",
)

c:\Users\Katie\miniconda3\envs\mltest\Lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
tokenizer_eng.save("tokenizer_Unigram_english.json")